# Technical Challenge Answers

## Database

1. The database was built according the [Tables Structure](https://github.com/p-beraldin/lending-data-engineer-test/blob/main/README.md#tables-structure) in the test guidelines.

2. The database was built with two tables, each one composed by the dataset `clients` and `loans`, provided in the guidelines, respectively.

The relation between these two tables are done by the key `user_id`, as a primary key in `clients` table and a foreign key in `loans`. This way, each loan is assigned to a user by its id.

```sql
CREATE TABLE clients (
	user_id bigint PRIMARY KEY NOT NULL,
	created_at timestamp NOT NULL,
	status varchar(50) NOT NULL,
	batch bigint NOT NULL,
	credit_limit bigint NOT NULL,
	interest_rate bigint NOT NULL,
	denied_reason text,
	denied_at timestamp
);

CREATE TABLE loans (
	loan_id bigint PRIMARY KEY NOT NULL,
	user_id bigint 
	created_at timestamp NOT NULL,
	due_at timestamp NOT NULL,
	paid_at timestamp,
	status varchar(50) NOT NULL,
	loan_amount decimal NOT NULL,
	tax decimal NOT NULL,
	due_amount decimal NOT NULL,
	amount_paid decimal NOT NULL,
	
	CONSTRAINT fk_client
		FOREIGN KEY(user_id) REFERENCES clients(user_id)
);
```


The data from the datasets were loaded into the tables with the following queries:
```sql
COPY clients(user_id, created_at, status, batch, credit_limit, interest_rate, denied_reason, denied_at)
FROM '<file location path>'
WITH DELIMITER ','
CSV HEADER;

COPY loans(user_id, loan_id, created_at, due_at, paid_at, status, loan_amount, tax, due_amount, amount_paid)
FROM '<file location path>'
WITH DELIMITER ','
CSV HEADER;
```

## SQL and Data Viz

1.

In [3]:
import pandas as pd
import psycopg2 as pg

engine = pg.connect("host=localhost port=5432 dbname=cloudwalk user=bruno password=bruno")

In [6]:
query_3_4 = """
SELECT TO_CHAR(created_at, 'YYYY-MM') AS month, COUNT(*) as number_of_loans, SUM(loan_amount) AS total_amount
FROM loans
GROUP BY 1;
"""

df_3_4 = pd.read_sql(query_3_4, con=engine)

/tmp/ipykernel_2157/3479404109.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3_4 = pd.read_sql(query_3_4, con=engine)


The DataFrame `df_3_4` contains the number of loans and total amount of loans by month. 

Top 10 months by number of loans:

In [9]:
df_3_by_loans = df_3_4.sort_values(by=['number_of_loans'], ascending=False)

df_3_by_loans.head(10)

,month,number_of_loans,total_amount
21,2023-12,17351,442464966.0
5,2024-01,16123,409112591.0
6,2023-11,13269,330839275.0
7,2023-10,11593,293005656.0
25,2023-09,8976,229573371.0
30,2023-08,7792,193164454.0
10,2023-07,6713,168555063.0
11,2023-06,5894,147375148.0
17,2023-05,5325,133772086.0
22,2023-04,4624,116296080.0


Top 10 months by amount:

In [12]:
df_3_by_amount = df_3_4.sort_values(by=['total_amount'], ascending=False)

df_3_by_amount.head(10)

,month,number_of_loans,total_amount
21,2023-12,17351,442464966.0
5,2024-01,16123,409112591.0
6,2023-11,13269,330839275.0
7,2023-10,11593,293005656.0
25,2023-09,8976,229573371.0
30,2023-08,7792,193164454.0
10,2023-07,6713,168555063.0
11,2023-06,5894,147375148.0
17,2023-05,5325,133772086.0
22,2023-04,4624,116296080.0


As we can see, the Top 10 of all months are equal in both cases.

2. Let's make the calculation of the adherence by batch (clients that got loans by total of clients in a batch)

In [14]:
query_adherence = """
    select c.batch as batch, count(distinct c.user_id)*100.0/count(c.user_id) as adherence from clients c 
    join loans l
    on c.user_id = l.user_id
    group by 1
    order by 2 desc;
"""
df_adherence = pd.read_sql(query_adherence, con=engine)
df_adherence

/tmp/ipykernel_2157/308092383.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_adherence = pd.read_sql(query_adherence, con=engine)


,batch,adherence
0,2,47.390084
1,3,47.298504
2,4,47.211522
3,1,46.939937


As we can see, all batches have, approximately, the same adherence, with rates varying from 46.9% to 47.3%. Batch 2 was the best one and 1 the worst one.

3. Let's calculate the default rate by interest rate.

Interest rate is given by subtracting from the due amount taxes and the loan amount (divided by the loan amount to calculate percentage).

In [16]:
query_default = """
    select 
	    round((due_amount-tax-loan_amount)*100/loan_amount, 1) as interest_rate,
	    (count(*) filter (where status = 'default'))*100.0/count(*) as default_rate
    from loans
    group by 1
    order by 2 desc;
"""
df_default = pd.read_sql(query_default, con=engine)
df_default

/tmp/ipykernel_2157/1749365740.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_default = pd.read_sql(query_default, con=engine)


,interest_rate,default_rate
0,22.5,8.256712
1,17.5,8.225585
2,7.5,8.222656
3,5.0,8.051026


As we can see from the above dataframe, there is a slightly higher default rate (8.26%) on 22.5% interest. 17.5 and 7.5% see equal default rate, and 5% rate sees a lower default rate of 8.05%, indicating that the lower interest rate shows a higher probability of payment, but not that higher, being just 0.2% higher and seing no increment after 7.5% rate. 

4. Let's first rank the best 10 clients. For a simple rank, let's consider that the best clients are the ones that pay the total amount of loans, or the most percentage of them. To avoid ties, let's give as a secondary rank rule the amount of money lent, since a person who pays 100% of a larger amount is expected to repeat the same pattern on a future loan. 

In [18]:
query_top10_clients = """
    select user_id, sum(amount_paid)*100/sum(due_amount) as payment_rate, sum(due_amount) as due_amount from loans
    group by 1
    order by 2 desc, 3 desc
    limit 10;
"""
df_top10_clients = pd.read_sql(query_top10_clients, con=engine)
df_top10_clients

/tmp/ipykernel_2157/1942169499.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_top10_clients = pd.read_sql(query_top10_clients, con=engine)


,user_id,payment_rate,due_amount
0,80867,100.0,416461.63
1,74414,100.0,413813.50
2,84194,100.0,405583.96
3,14050,100.0,403217.18
4,8950,100.0,381074.56
5,71171,100.0,355880.15
6,16710,100.0,351265.85
7,32391,100.0,349722.75
8,2473,100.0,345422.07
9,26637,100.0,341975.09


We can use the same reasoning for the worst 10 clients, but it is unlikely to have ties on the payment rate, but we'll keep them on the query for the sake of data analysis.

In [20]:
query_worst10_clients = """
    select user_id, sum(amount_paid)*100/sum(due_amount) as payment_rate, sum(due_amount) as due_amount from loans
    group by 1
    order by 2, 3
    limit 10;
"""
df_worst10_clients = pd.read_sql(query_worst10_clients, con=engine)
df_worst10_clients

/tmp/ipykernel_2157/4190710623.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_worst10_clients = pd.read_sql(query_worst10_clients, con=engine)


,user_id,payment_rate,due_amount
0,52295,0.000430,4651.02
1,18502,0.013430,1638.11
2,73736,0.019131,37373.43
3,55008,0.026958,4228.80
4,44494,0.036100,37285.20
5,16387,0.036592,13992.18
6,38103,0.039801,96430.69
7,82131,0.041015,44861.33
8,52023,0.043832,59363.47
9,3737,0.048172,12310.18


This ranking system is considerably simple and does not consider the aging of a loan to be less important than a more recent loan. As an improvement example from another approach more focused on a machine learning environment, a Naive Bayes algorithm to calculate the probability of default of a future loan given all the data on the database as a training set would be a suitable ranking to determine the best and worst clients. The metric would be only the probability of default, meaning that a lower probability would correspond to a better client. This can even lead to a trigger rule to avoid new loans from a client if this probability of default would pass a determined treshold.

5. Let's calculate the default rate, just like we did before, but grouped by month and batch. 

In [22]:
query_default_by_month_batch = """
    select to_char(l.created_at, 'YYYY-MM') as month, c.batch as batch, (count(*) filter (where l.status = 'default'))*100.0/count(*) as default_rate from loans l
    join clients c 
    on l.user_id = c.user_id
    group by 1, 2
    order by 1, 2;
"""

df_default_by_month_batch = pd.read_sql(query_default_by_month_batch, con=engine)
df_default_by_month_batch.head()



/tmp/ipykernel_2157/2379545212.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_default_by_month_batch = pd.read_sql(query_default_by_month_batch, con=engine)


,month,batch,default_rate
0,2020-01,1,8.333333
1,2020-01,2,33.333333
2,2020-01,3,100.000000
3,2020-02,1,13.888889
4,2020-02,2,15.000000


6. 